<h1>PAGERANK</h1>

In [201]:
import numpy as np
import pandas as pd
import nltk as nltk
import math
import operator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/manorlf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [202]:
data_set = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
data_set.columns = ['source', 'target', 'rating', 'time']
data_set.head()


,source,target,rating,time
0,6,5,2,1.289242e+09
1,1,15,1,1.289243e+09
2,4,3,7,1.289245e+09
3,13,16,8,1.289254e+09
4,13,10,8,1.289254e+09


In [203]:
#Filtering The ones with rating grater or iqual to 8

new_data = data_set.loc[data_set['rating'] >= 8]
new_data.index = np.arange(0, len(new_data))


In [204]:
#Mapping sorce and target to a list
matter_inverstors = list(set(new_data.source).union(set(new_data.target)))


In [205]:
adj_dict = dict()

In [206]:
def fill_adj(adj_dict, data_set, matter_inverstors):
    for index in range(data_set.shape[0]):
        add_to_adj(adj_dict, matter_inverstors.index(data_set.loc[index,'source']), matter_inverstors.index(data_set.loc[index,'target']))
        
def add_to_adj(adj, source, target):
    if (source in adj):
        adj[source].append(target)
    else:
        adj[source] = [target]
    
    if (target not in adj):
        adj[target] = []


In [207]:
def fill_transition_matrix(transition_matrix, adj_dict):
    for node in adj_dict.keys():
        adjacents = adj_dict[node]
        if (len(adjacents) > 0):
            weight = 1.0/len(adjacents)
            for adj in adjacents:
                transition_matrix[adj][node] = weight


In [208]:
#Constants
TOTAL_NODES = len(matter_inverstors)

In [209]:
fill_adj(adj_dict, new_data, matter_inverstors)

In [210]:
'''
    Original Transition Matrix
'''

transition_matrix = np.zeros((TOTAL_NODES, TOTAL_NODES))

In [211]:
'''
    Filling the values for each node that is linked with others
'''
fill_transition_matrix(transition_matrix, adj_dict)

In [212]:
'''
    Google Matrix m with dumping factor 0.15
'''
dumping_matrix_factor = np.ones((TOTAL_NODES, TOTAL_NODES))
dumping_matrix_factor = (1.0/float(TOTAL_NODES) * np.matrix(dumping_matrix_factor))


In [213]:
m = 0.85 * transition_matrix + 0.15 * dumping_matrix_factor

In [214]:
'''
    Original Normalized Vector
'''
normalized_vector = np.ones((TOTAL_NODES, 1))
normalized_vector = (1.0/float(TOTAL_NODES) * np.matrix(normalized_vector))

In [215]:
cont = 0

def pagerank(v):
    global cont
    calc = sum(abs(m * v - v))
    if (calc > 0.001):
        cont += 1
        return pagerank(m * v)
    else:
        cont += 1
        return m * v
    
    

In [216]:
v = normalized_vector
result = pagerank(v)



In [225]:
print(cont)
result_list = []
for i in result:
    result_list.append(i.item())


35


In [228]:
ranked_df = pd.DataFrame({'nodes':matter_inverstors, 'pagerank':result_list})
ranked_df.head()

ranked_df.sort_values('pagerank', ascending=False).take(range(5))

,nodes,pagerank
0,1,0.000139
120,202,0.000125
94,144,0.000117
887,3996,0.000093
182,361,0.000092


In [231]:
ranked_df.to_csv(path_or_buf='results/result.csv', index=False)
new_data.to_csv(path_or_buf='results/filtered-sign-bitcoinotc.csv', columns=['source', 'target', 'rating'], index=False)